In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [48]:
    # importation
    import pandas as pd
    !pip install easy_dna

# Load the dataset

In [90]:
Xtr0 = pd.read_csv("./data/Xtr0.csv", sep=",", index_col=0)
Xtr1 = pd.read_csv("./data/Xtr1.csv", sep=",", index_col=0).values
Xtr2 = pd.read_csv("./data/Xtr2.csv", sep=",", index_col=0).values

# shape (2000,100): float
Xtr0_mat100 = pd.read_csv("./data/Xtr0_mat100.csv", sep=" ", header=None).values
Xtr1_mat100 = pd.read_csv("./data/Xtr1_mat100.csv", sep=" ", header=None).values
Xtr2_mat100 = pd.read_csv("./data/Xtr2_mat100.csv", sep=" ", header=None).values

# shape (2000,1): string
Xte0 = pd.read_csv("./data/Xte0.csv", sep=",", index_col=0).values
Xte1 = pd.read_csv("./data/Xte1.csv", sep=",", index_col=0).values
Xte2 = pd.read_csv("./data/Xte2.csv", sep=",", index_col=0).values

# shape (2000,100): float
Xte0_mat100 = pd.read_csv("./data/Xte0_mat100.csv", sep=" ", header=None).values
Xte1_mat100 = pd.read_csv("./data/Xte1_mat100.csv", sep=" ", header=None).values
Xte2_mat100 = pd.read_csv("./data/Xte2_mat100.csv", sep=" ", header=None).values

# shape (2000,1): 0 or 1
Ytr0 = pd.read_csv("./data/Ytr0.csv", sep=",", index_col=0).values
Ytr1 = pd.read_csv("./data/Ytr1.csv", sep=",", index_col=0).values
Ytr2 = pd.read_csv("./data/Ytr2.csv", sep=",", index_col=0).values

#let us print the sequence 929

#print(f" the sequence 929 has a length of  {length} " )

#Rescaling labels
Ytr0 = np.where(Ytr0 == 0, -1, 1)
Ytr1 = np.where(Ytr1 == 0, -1, 1)
Ytr2 = np.where(Ytr2 == 0, -1, 1)


In [91]:
Xtr0.head()

,seq
Id,
0,TCCTGTGCACATCTGCACCCCTGTTGTGGCCACAAAATGATCCGGC...
1,TTAAGTGTATATCTAATAATTTTTTTGCCTACATTCCTGTGTTACC...
2,GTGCTCAATTAGTTGCCTACAAATAGTAGCCTGGCACAGTGTAAGC...
3,CACCTGGAAAATACAAACAGGCGCAAGAAGAGTTAACCCACAGATC...
4,AAATCACTGCCTATCCTTGGGCCAAAAGGTTTCTACAGGAAGCTGC...


In [92]:
#Xtr0.iloc[0]['seq']
Xtr0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   seq     2000 non-null   object
dtypes: object(1)
memory usage: 31.2+ KB


In [93]:
Xtr0.describe()

,seq
count,2000
unique,2000
top,TCCTGTGCACATCTGCACCCCTGTTGTGGCCACAAAATGATCCGGC...
freq,1


# Dna preprocessing

In [94]:
#   link for : https://www.theaidream.com/post/demystify-dna-sequencing-with-machine-learning-and-python

def Kmers_funct(seq, size=1):
   return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]
#convert our training data sequences into short overlapping k-mers of length 6. Lets do that for each species of data we have using our Kmers_funct function.

Xtr0['words'] = Xtr0.apply(lambda x: Kmers_funct(x.loc['seq']), axis=1)
Xtr0 = Xtr0.drop('seq', axis=1)
Xtr0.head()


,words
Id,
0,"[t, c, c, t, g, t, g, c, a, c, a, t, c, t, g, ..."
1,"[t, t, a, a, g, t, g, t, a, t, a, t, c, t, a, ..."
2,"[g, t, g, c, t, c, a, a, t, t, a, g, t, t, g, ..."
3,"[c, a, c, c, t, g, g, a, a, a, a, t, a, c, a, ..."
4,"[a, a, a, t, c, a, c, t, g, c, c, t, a, t, c, ..."


In [97]:


#transpose the DataFrame
Xtr0 = Xtr0.transpose()
Xtr0.describe()

,words
count,2000
unique,2000
top,"[t, c, c, t, g, t, g, c, a, c, a, t, c, t, g, ..."
freq,1


In [53]:
#We need to now convert the lists of k-mers for each gene into string sentences of words that can be used to create the Bag of Words model. We will make a target variable y to hold the class labels.

'''''
human_texts = list(Xtr0['words'])
for item in range(len(human_texts)):
   human_texts[item] = ' '.join(human_texts[item])
human_texts
X=[]
for i in range (len(human_texts)):
    X.append(human_texts[i].split())
X[0][1]
,,,,

'cctgtgcacatc'

In [54]:
# word2vec

!pip install nltk
!pip install gensim

In [55]:
import gensim
from gensim.models import Word2Vec
model_word_2vec = Word2Vec(X,vector_size =100, min_count = 1)
#print(model_word_2vec.wv(X[0][1]))
print(len(X[0]))
X_embd=np.zeros((len(X),len(X[0]*100)))
for i in range(len(X)):
    for j in range(len(X[0])):
        word=X[i][j]
        
        X_embd[i,j:j+100]=np.array(model_word_2vec.wv[word])
    
    
        
X_embd.shape        

90


(2000, 9000)

In [56]:
#Convert our k-mer words into uniform length numerical vectors that represent counts for every k-mer in the vocabulary:
#from sklearn.feature_extraction.text import CountVectorizer
#cv = CountVectorizer() #The n-gram size of 4 is previously determined by testing
#X = cv.fit_transform(human_texts)
#X=X.toarray()
#print(X)
#print(cv.get_feature_names_out)

A transcription is a process of making a copy of an gene DNA sequence .this copy is called mRNA(messager RNA).
This  project is a sequence classification task, predicting weather a DNA sequence is binding to a specific transcritor fator.

some Kernels implementation
-here we will implement some kernels like linear kernels,
gaussian kernels

In [57]:
class Kernel():
    def __init__(self):
        pass

    def similarity(self, x, y, **kwargs):
        """ Similarity between 2 feature vectors (depends on the type of kernel)"""
        return -1

    def gram_matrix(self, X1, X2,**kwargs):
        """ Compute the gram matrix of a data vector X where the (i,j) entry is defined as <Xi,Xj>\\
        X1: data vector (n_samples_1 x n_features)
        X2: data vector (n_samples_2 x n_features), if None compute the gram matrix for (X1,X1)
        """
        if X2 is None: 
            X2=X1
        n_samples_1 = X1.shape[0]
        n_samples_2 = X2.shape[0]
        K = np.zeros((n_samples_1, n_samples_2))
        for i in range(n_samples_1):
            for j in range(n_samples_2):
                K[i,j] = self.similarity(X1[i],X2[j],**kwargs)
        return K

    


In [58]:
class LinearKernels(Kernel):
    def __init():
        super().__init__()
    
    def similarity(self,X1,X2,**kwargs):
        
        return X1.T@X2
    
    
    
 

In [59]:
class GaussianKernel(Kernel):

    def __init__(self):
        super().__init__()
        

    def similarity(self, x, y,**kwargs):
        """ gaussian kernel : k(x,y) = 1/ sqrt(2 pi sigma2)^n * exp( - ||x-y||^2 / 2 sigma^2 )\\
        x, y: array (n_features,)
        """
        sigma=kwargs.get('sigma')
        normalize=kwargs.get('normalize')
        
        if normalize:
            norm_fact = (np.sqrt(2 * np.pi) * sigma) ** len(x)
            return np.exp(-np.linalg.norm(x-y)**2 / (2 *sigma**2)) / norm_fact
        else:
            return np.exp(-np.linalg.norm(x-y)**2 / (2 * sigma**2))



In [60]:
class PolynomialKernel(Kernel):

    def __init__(self):
        super().__init__()
        

    def similarity(self, x, y, **kwargs):
        gamma=kwargs.get('gamma')
        degree=kwargs.get('degree')
        coef=kwargs.get('coef')
        coef0=kwargs.get('coef0')
        """ polynomial kernel : k(x,y) = (gamma <x,y> + r)^d \\
        x, y: array (n_features,)
        """
        return (gamma * np.dot(x,y) + coef0)**degree 


    

In [61]:
# polynomial and gaussian


class PolynomialKernelGaussian(Kernel):

    def __init__(self):
        print("entre dans polyGaussian")
        super().__init__()
        

    def similarity(self, x, y, **kwargs):
        
        gamma=kwargs.get('gamma')
        degree=kwargs.get('degree')
        coef=kwargs.get('coef')
        coef0=kwargs.get('coef0')
        """ polynomial kernel : k(x,y) = (gamma <x,y> + r)^d \\
        x, y: array (n_features,)
        """
        return  0.5*((gamma * np.dot(x,y) + coef0)**degree) + 0.8*(np.exp(-np.linalg.norm(x-y)**2 / (2 * sigma**2)))+2*(x.T@y)


In [62]:
class SpectrumKernel(Kernel):

    def __init__(self):
        super().__init__()
       

    def similarity(self, x, y,**kwargs):
        """ Spectrum kernel \\
        x, y: string
        """
        k=kwargs.get('k')
        substr_x, counts_x = np.unique([x[i:i+k] for i in range(len(x)-k+1)], return_counts=True)
        return np.sum(np.char.count(y, substr_x)*counts_x)


SVM implementation 


# ****HERE WE MAKE A CLASSIFICATION USING SVM KERNALIZATION****

In [63]:
class KernelMethodBase(object):
    '''
    Base class for kernel methods models

    Methods
    ----
    fit
    predict
    fit_K
    predict_K
    '''
    kernels_ = {
        'linear': LinearKernels(),
        'rbf': GaussianKernel(),
        'poly':PolynomialKernel(),
        'poly_gaussian':PolynomialKernelGaussian(),
        'spectrum': SpectrumKernel(),
    }
    
        # 'mismatch': mismatch_kernel,
    
    def __init__(self, kernel='linear', **kwargs):
        print(kernel)
        self.kernel_name = kernel
        self.kernel_function_ = self.kernels_[kernel].gram_matrix
        self.kernel_parameters = self.get_kernel_parameters(**kwargs)
        self.fit_intercept_ = False

    def get_kernel_parameters(self, **kwargs):
        #'linear' 'rbf', 'poly', 'spectrum', 'mismatch' or 'sum' 
        gamma = 10.0
        sigma=np.sqrt(0.5/gamma)
        C = 5.0 #Parameter C for SVM
         #Parameter gamma for SVM (only for 'rbf' or 'poly')
        coef0 = 1.0 #Parameter coef0 for SVM (only for 'poly')
        degree = 3 
        params = {}
        if self.kernel_name == 'rbf':
            params['sigma'] = kwargs.get('sigma', sigma)
            params['normalize']=kwargs.get('normalize', True)
        if self.kernel_name == 'poly':
            params['degree'] = kwargs.get('degree', 10)
            params['coef'] = kwargs.get('coef', 5)
            params['coef0'] = kwargs.get('coef0', 1)
            params['gamma'] = kwargs.get('gamma', 10)
        if self.kernel_name == 'poly_gaussian':
            params['sigma'] = kwargs.get('sigma', sigma)
            params['degree'] = kwargs.get('degree', 3)
            params['coef'] = kwargs.get('coef', 3)
            params['coef0'] = kwargs.get('coef0', 1)
            params['gamma'] = kwargs.get('gamma', 10)
        if self.kernel_name == 'spectrum':
            params['k'] = kwargs.get('k', 12)
            
        return params

    def fit_K(self, K, y, **kwargs):
        """Finds optimal parameters from K(X_train, X_train) and y."""
        pass
        


    def decision_function_K(self, K):
        """Returns decision values from K(X, X_train)."""
        pass

    def fit(self, X, y, fit_intercept=False, **kwargs):

        if fit_intercept:
            X = add_column_ones(X)
            self.fit_intercept_ = True
        self.X_train = X
        self.y_train = y

        K = self.kernel_function_(self.X_train, self.X_train, **self.kernel_parameters)
        print(f'shape of k{K.shape}')
    

        return self.fit_K(K, y, **kwargs)

    def decision_function(self, X):

        if self.fit_intercept_:
            X = add_column_ones(X)

        K_x = self.kernel_function_(X, self.X_train, **self.kernel_parameters)
        print(f'shape of kx{K_x.shape}')
       
        return self.decision_function_K(K_x)

    def predict(self, X):
        """Predicts the labels (e.g. +/- 1 for classification) using the decision function."""
        pass
    
    def predict_K(self, K):
        """Predicts the labels (e.g. +/- 1 for classification) using the decision function."""
        pass

entre dans polyGaussian


In [64]:
!pip install cvxopt
import cvxopt

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M.astype(np.double)) if M is not None else None for M in [P, q, G, h, A, b]
    ]
    #cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(*cvx_matrices, options={'show_progress': False})
    return np.array(solution['x']).flatten()

solve_qp=cvxopt_qp


In [65]:
def svm_dual_soft_to_qp_kernel(K, y, C=1):
    n = K.shape[0]
    assert (len(y) == n)
    
    # Dual formulation, soft margin
    #print(y)
    print( (np.diag(y.squeeze())))
    P =(np.diag(y.squeeze()))@K@(np.diag(y.squeeze()))

    # As a regularization, we add epsilon * identity to P
    eps = 1e-12
    P += eps * np.eye(n)
    q = - np.ones(n)
    G = np.vstack([-np.eye(n), np.eye(n)])
    h = np.hstack([np.zeros(n), C * np.ones(n)])
    A = y.squeeze()[np.newaxis, :]
    b = np.array([0.])
  

    
    print(f"shape of P{P.shape}")
    print(f"shape of q{q.shape}")
    print(f"shape of G{G.shape}")
    print(f"shape of h{h.shape}")
    print(f"shape of A{A.shape}")
    print(f"shape of b{b.shape}")
    
    return P, q, G, h, A, b


In [66]:
class KernelSVM(KernelMethodBase):
    '''
    Kernel SVM Classification
    
    Methods
    ----
    fit
    predict
    '''
    def __init__(self, C=0.1, **kwargs):
        self.C = C
        super().__init__(**kwargs)
    
    def fit_K(self, K, y, tol=1e-3):
        # Solve dual problem
        self.alpha = solve_qp(*svm_dual_soft_to_qp_kernel(K, y, C=self.C))
        
        
        print(f'shape of alpha{self.alpha.shape}')
        # Compute support vectors and bias b
        sv = np.logical_and((self.alpha > tol), (self.C - self.alpha > tol))
        self.bias = y[sv] - K[sv].dot(self.alpha * y)
        self.bias = self.bias.mean()

        self.support_vector_indices = np.nonzero(sv)[0]
        
        return self
        
    def decision_function_K(self, K_x):
        print(f'alphay {(self.alpha*self.y_train).shape}')
        return K_x.dot(self.alpha.reshape(-1,1)*self.y_train.reshape(-1,1)) + self.bias

    def predict(self, X):
         return np.sign(self.decision_function(X))

# ****Hyperparameters tunning****

In [67]:
# Prediction error
def error(ypred, ytrue):
    e = (ypred != ytrue).mean()
    return e

def add_column_ones(X):
    n = X.shape[0]
    return np.hstack([X, np.ones((n, 1))])

In [68]:
print(Xte0_mat100.shape)

(1000, 100)


# prediction with k-mer

In [83]:
import tqdm as tqdm
np.random.seed(42)
kernel = 'poly_gaussian'

sigma = 0.8
gamma=9
degree =2
C =0.01
k=12
#tol = 1e-3
tol = 1e-3
#model0 = KernelSVM(C=C, kernel=kernel)

model0 = KernelSVM(C=C, kernel=kernel,degree=degree,sigma=np.sqrt(0.5/gamma),gamma=gamma)


print(Ytr0.shape)

n,d=X_embd.shape
idx=np.arange(n)
idx=np.random.permutation(idx)
print(idx)
X_embd=X_embd[idx]
Ytr0=Ytr0[idx]
x=X_embd[:int(0.8*n)]

print(x.shape)
#y=np.where(Ytr0[int(0.8*n):]==0,-1,1)
y=Ytr0[:int(0.8*n)]
print(y.shape)
model0.fit(x,y)
ypred0=model0.predict(X_embd[int(0.8*n):])
#plot_decision_function(model, Xtr1, Ytr1,
                      # title='SVM {} Kernel'.format(kernel))
print('Test error: {:.2%}'.format(error(ypred0, Ytr0[int(0.8*n):])))


print(ypred0.shape)

poly_gaussian
(2000, 1)
[1860  353 1333 ...  860 1459 1126]
(1600, 9000)
(1600, 1)
shape of k(1600, 1600)
[[ 1  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 0  0 -1 ...  0  0  0]
 ...
 [ 0  0  0 ... -1  0  0]
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0  0  1]]
shape of P(1600, 1600)
shape of q(1600,)
shape of G(3200, 1600)
shape of h(3200,)
shape of A(1, 1600)
shape of b(1,)
shape of alpha(1600,)
shape of kx(400, 1600)
alphay (1600, 1600)
Test error: 46.50%
(400, 1)


In [46]:
import tqdm as tqdm

kernel = 'poly_gaussian'

#sigma = 0.8
gamma=9
degree =2
C =0.01
k=12
tol = 1e-3
#model0 = KernelSVM(C=C, kernel=kernel)

model0 = KernelSVM(C=C, kernel=kernel,degree=degree,sigma=np.sqrt(0.5/gamma),gamma=gamma)

ratio=5
n,p=Xtr0_mat100.shape
print(Ytr0.shape)
print(Xtr0.shape)
x=Xtr0_mat100[:int(0.8*n)]
#y=np.where(Ytr0[int(0.8*n):]==0,-1,1)
y=Ytr0[:int(0.8*n)]
model0.fit(x,y)
ypred0=model0.predict(Xtr0_mat100[int(0.8*n):])
#ypred0=np.where(ypred0== 0,-1,1)
#plot_decision_function(model, Xtr1, Ytr1,
                      # title='SVM {} Kernel'.format(kernel))
print('Test error: {:.2%}'.format(error(ypred0, Ytr0[int(0.8*n):])))


print(ypred0.shape)

poly_gaussian
(2000, 1)
(2000, 1)
shape of k(1600, 1600)
[[-1  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 0  0 -1 ...  0  0  0]
 ...
 [ 0  0  0 ... -1  0  0]
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0 -1]]
shape of P(1600, 1600)
shape of q(1600,)
shape of G(3200, 1600)
shape of h(3200,)
shape of A(1, 1600)
shape of b(1,)


KeyboardInterrupt: 

# prediction on the dataset1

In [503]:
kernel = 'poly_gaussian'

#sigma = 0.8
gamma=9
degree =2
C =0.1
k=12
tol = 1e-3
#model0 = KernelSVM(C=C, kernel=kernel)

model1 = KernelSVM(C=C, kernel=kernel,degree=degree,sigma=np.sqrt(0.5/gamma),gamma=gamma)

ratio=5
#model0 = KernelSVM(C=C, kernel=kernel,degree=degree,sigma=np.sqrt(0.5/gamma))

#model = KernelSVM(C=C, kernel=kernel,degree=degree)
ratio=0.5
n,p=Xtr0_mat100.shape
print(Ytr0.shape)
print(Xte0_mat100.shape)

model1.fit(Xtr0_mat100, Ytr0)
ypred1=model1.predict(Xte0_mat100)
ypred1=np.where(ypred1==-1,0,1)
y1=np.where(Ytr0==-1,0,1)
print("arrive")
print(ypred1.shape)
#plot_decision_function(model, Xtr1, Ytr1,
                      # title='SVM {} Kernel'.format(kernel))
#print('Test error: {:.2%}'.format(error(ypred1, y1)))

poly_gaussian
(2000, 1)
(1000, 100)
shape of k(2000, 2000)
[[-1  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ... -1  0  0]
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0  0 -1]]
shape of P(2000, 2000)
shape of q(2000,)
shape of G(4000, 2000)
shape of h(4000,)
shape of A(1, 2000)
shape of b(1,)
shape of alpha(2000,)
shape of kx(1000, 2000)
alphay (2000, 2000)
arrive
(1000, 1)


# prediction dataset2

In [498]:
kernel = 'poly_gaussian'

#sigma = 0.8
gamma=9
degree =2
C =0.1
k=12
tol = 1e-3
#model0 = KernelSVM(C=C, kernel=kernel)

model2 = KernelSVM(C=C, kernel=kernel,degree=degree,sigma=np.sqrt(0.5/gamma),gamma=gamma)

ratio=5
#model = KernelSVM(C=C, kernel=kernel,degree=degree)
ratio=0.5
n,p=Xte1_mat100.shape
print(Ytr1.shape)
print(Xtr1_mat100.shape)

model2.fit(Xtr1_mat100, Ytr1)
#y_pred = model2.predict(Xtr1_mat100[int(0.8*n):])
#y_pred=np.where(y_pred == -1, 0, 1)
#y2=np.where(Ytr1[int(0.8*n):]==-1,0,1)
#plot_decision_function(model, Xtr1, Ytr1,
                      # title='SVM {} Kernel'.format(kernel))
#print('Test error: {:.2%}'.format(error(y_pred, y2)))

ypred2=model2.predict(Xte1_mat100)
ypred2=np.where(ypred2==-1,0,1)
y2=np.where(Ytr1==-1,0,1)
print(ypred1.shape)

poly_gaussian
(2000, 1)
(2000, 100)
shape of k(2000, 2000)
[[-1  0  0 ...  0  0  0]
 [ 0 -1  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0  1]]
shape of P(2000, 2000)
shape of q(2000,)
shape of G(4000, 2000)
shape of h(4000,)
shape of A(1, 2000)
shape of b(1,)
shape of alpha(2000,)
shape of kx(1000, 2000)
alphay (2000, 2000)
(1000, 1)


# prediction dataset 3

In [499]:
import tqdm as tqdm
kernel = 'poly_gaussian'

#sigma = 0.8
gamma=9
degree =2
C =0.1
k=12
tol = 1e-3
#model0 = KernelSVM(C=C, kernel=kernel)

model3 = KernelSVM(C=C, kernel=kernel,degree=degree,sigma=np.sqrt(0.5/gamma),gamma=gamma)

ratio=5
#model = KernelSVM(C=C, kernel=kernel,degree=degree)
ratio=0.5
n,p=Xtr2_mat100.shape
print(Ytr2.shape)
print(Xtr2_mat100.shape)

model3.fit(Xtr2_mat100, Ytr2)
#y_pred = model3.predict(Xtr2_mat100[int(0.8*n):])
#y_pred=np.where(y_pred == -1, 0, 1)
#y3=np.where(Ytr2[int(0.8*n):]==-1,0,1)
#plot_decision_function(model, Xtr1, Ytr1,
                      # title='SVM {} Kernel'.format(kernel))
#print('Test error: {:.2%}'.format(error(y_pred, y3)))

ypred3=model3.predict(Xte2_mat100)
ypred3=np.where(ypred3==-1,0,1)
y3=np.where(Ytr2==-1,0,1)
print(ypred3.shape)

poly_gaussian
(2000, 1)
(2000, 100)
shape of k(2000, 2000)
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
shape of P(2000, 2000)
shape of q(2000,)
shape of G(4000, 2000)
shape of h(4000,)
shape of A(1, 2000)
shape of b(1,)
shape of alpha(2000,)
shape of kx(1000, 2000)
alphay (2000, 2000)
(1000, 1)


In [500]:

#ypred2= np.where(model2.predict(Xte1_mat100)== -1,0,1)
#ypred3= np.where(model3.predict(Xte2_mat100)== -1,0,1)


In [501]:

pred = np.concatenate([ypred1.squeeze(),ypred2.squeeze(),ypred3.squeeze()])
pred_df = pd.DataFrame()
print(pred.shape)
print(pred)
pred_df['Bound'] = pred
pred_df.index.name = 'Id'
pred_df.to_csv('Yte_poly_gaussian_linear_kernel.csv', sep=',', header=True)


(3000,)
[1 1 1 ... 1 0 1]
